In this notebook, we will use Callbacks during training to save a model in .h5 extension so we can load and use at a later time.
Using callbacks is very handy when you have large dataset that takes hours to train the model and you have to save each model after an epoch. 

This is also helpful in cases of unexpected interruption, so we wont lose th eentire progress of training.

# Using callbacks during Training

### Data Preparation

In [1]:
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
# %matplotlib inline

from sklearn.datasets import fetch_california_housing
housing = fetch_california_housing()

print(housing.feature_names)

from sklearn.model_selection import train_test_split
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target, test_size = 0.2, random_state = 42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, test_size = 0.3, random_state=42)


from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

np.random.seed(42)
tf.random.set_seed(42)

['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']


### Create a model

In [2]:
tf.keras.backend.clear_session()

input_ = tf.keras.layers.Input(shape=X_train.shape[1:])
hidden1 = tf.keras.layers.Dense(30, activation='relu')(input_)
hidden2 = tf.keras.layers.Dense(30, activation='relu')(hidden1)
concat = tf.keras.layers.concatenate([input_, hidden2])
output = tf.keras.layers.Dense(1)(concat)
model_functional = tf.keras.models.Model(inputs=[input_], outputs=[output])

model_functional.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 8)]          0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 30)           270         input_1[0][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 30)           930         dense[0][0]                      
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 38)           0           input_1[0][0]                    
                                                                 dense_1[0][0]         

In [3]:
model_functional.compile(loss='mse',
                         optimizer=tf.keras.optimizers.SGD(lr=1e-3),
                        metrics=['mae'])

In [4]:
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint('Model_Func_{epoch:02d}.h5')

In [5]:
model_history = model_functional.fit(X_train, y_train, epochs=10,
                                     validation_data=(X_valid, y_valid),
                                    callbacks=[checkpoint_cb])

Epoch 1/10
362/362 [==============================] - 0s 901us/step - loss: 1.2605 - mae: 0.7982 - val_loss: 0.7210 - val_mae: 0.6104
Epoch 2/10
362/362 [==============================] - 0s 633us/step - loss: 0.6456 - mae: 0.5819 - val_loss: 0.6469 - val_mae: 0.5727
Epoch 3/10
362/362 [==============================] - 0s 617us/step - loss: 0.6041 - mae: 0.5605 - val_loss: 0.6127 - val_mae: 0.5564
Epoch 4/10
362/362 [==============================] - 0s 625us/step - loss: 0.5759 - mae: 0.5465 - val_loss: 0.5814 - val_mae: 0.5418
Epoch 5/10
362/362 [==============================] - 0s 634us/step - loss: 0.5538 - mae: 0.5349 - val_loss: 0.5646 - val_mae: 0.5332
Epoch 6/10
362/362 [==============================] - 0s 625us/step - loss: 0.5369 - mae: 0.5259 - val_loss: 0.5500 - val_mae: 0.5272
Epoch 7/10
362/362 [==============================] - 0s 627us/step - loss: 0.5206 - mae: 0.5189 - val_loss: 0.5288 - val_mae: 0.5141
Epoch 8/10
362/362 [==============================] - 0s 628us

In [6]:
del model_functional
tf.keras.backend.clear_session()

In [8]:
model = tf.keras.models.load_model('Model_Func_10.h5')
mse_test = model.evaluate(X_test, y_test)

129/129 [==============================] - 0s 374us/step - loss: 0.5152 - mean_absolute_error: 0.5055


# Create callback for only best model

In [10]:
tf.keras.backend.clear_session()

input_ = tf.keras.layers.Input(shape=X_train.shape[1:])
hidden1 = tf.keras.layers.Dense(30, activation='relu')(input_)
hidden2 = tf.keras.layers.Dense(30, activation='relu')(hidden1)
concat = tf.keras.layers.concatenate([input_, hidden2])
output = tf.keras.layers.Dense(1)(concat)
model_functional = tf.keras.models.Model(inputs=[input_], outputs=[output])

model_functional.compile(loss='mse',
                         optimizer=tf.keras.optimizers.SGD(lr=1e-3),
                        metrics=['mae'])

checkpoint_cb = tf.keras.callbacks.ModelCheckpoint('Model_Func_Best.h5', save_best_only=True)

In [11]:
model_history = model_functional.fit(X_train, y_train, epochs=10,
                                     validation_data=(X_valid, y_valid),
                                    callbacks=[checkpoint_cb])

Epoch 1/10
362/362 [==============================] - 0s 806us/step - loss: 1.4786 - mae: 0.8893 - val_loss: 0.8711 - val_mae: 0.6874
Epoch 2/10
362/362 [==============================] - 0s 638us/step - loss: 0.8012 - mae: 0.6551 - val_loss: 0.7782 - val_mae: 0.6446
Epoch 3/10
362/362 [==============================] - 0s 630us/step - loss: 0.7335 - mae: 0.6235 - val_loss: 0.7165 - val_mae: 0.6159
Epoch 4/10
362/362 [==============================] - 0s 624us/step - loss: 0.6862 - mae: 0.6008 - val_loss: 0.6784 - val_mae: 0.5945
Epoch 5/10
362/362 [==============================] - 0s 630us/step - loss: 0.6502 - mae: 0.5828 - val_loss: 0.6638 - val_mae: 0.5799
Epoch 6/10
362/362 [==============================] - 0s 626us/step - loss: 0.6241 - mae: 0.5702 - val_loss: 0.6172 - val_mae: 0.5678
Epoch 7/10
362/362 [==============================] - 0s 625us/step - loss: 0.5938 - mae: 0.5582 - val_loss: 0.5908 - val_mae: 0.5493
Epoch 8/10
362/362 [==============================] - 0s 618us

# Get and use the best model

In [12]:
del model_functional
tf.keras.backend.clear_session()

In [13]:
model = tf.keras.models.load_model('Model_Func_Best.h5')
mse_test = model.evaluate(X_test, y_test)

129/129 [==============================] - 0s 364us/step - loss: 0.5603 - mean_absolute_error: 0.5295


# Create best model using early stopping

In [17]:
tf.keras.backend.clear_session()

input_ = tf.keras.layers.Input(shape=X_train.shape[1:])
hidden1 = tf.keras.layers.Dense(30, activation='relu')(input_)
hidden2 = tf.keras.layers.Dense(30, activation='relu')(hidden1)
concat = tf.keras.layers.concatenate([input_, hidden2])
output = tf.keras.layers.Dense(1)(concat)
model_functional = tf.keras.models.Model(inputs=[input_], outputs=[output])

model_functional.compile(loss='mse',
                         optimizer=tf.keras.optimizers.SGD(lr=1e-3),
                        metrics=['mae'])

checkpoint_cb = tf.keras.callbacks.ModelCheckpoint('Early_Stop_Model.h5', save_best_only=True)

In [19]:
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)

In [20]:
model_history = model_functional.fit(X_train, y_train, epochs=10,
                                     validation_data=(X_valid, y_valid),
                                    callbacks=[checkpoint_cb, early_stopping_cb])

Epoch 1/10
362/362 [==============================] - 0s 805us/step - loss: 1.8534 - mae: 0.9494 - val_loss: 0.8408 - val_mae: 0.6768
Epoch 2/10
362/362 [==============================] - 0s 635us/step - loss: 0.7894 - mae: 0.6552 - val_loss: 0.7520 - val_mae: 0.6382
Epoch 3/10
362/362 [==============================] - 0s 643us/step - loss: 0.7213 - mae: 0.6260 - val_loss: 0.6971 - val_mae: 0.6132
Epoch 4/10
362/362 [==============================] - 0s 631us/step - loss: 0.6725 - mae: 0.6028 - val_loss: 0.6572 - val_mae: 0.5922
Epoch 5/10
362/362 [==============================] - 0s 635us/step - loss: 0.6359 - mae: 0.5846 - val_loss: 0.6291 - val_mae: 0.5777
Epoch 6/10
362/362 [==============================] - 0s 634us/step - loss: 0.6082 - mae: 0.5704 - val_loss: 0.5977 - val_mae: 0.5650
Epoch 7/10
362/362 [==============================] - 0s 633us/step - loss: 0.5825 - mae: 0.5590 - val_loss: 0.5765 - val_mae: 0.5488
Epoch 8/10
362/362 [==============================] - 0s 614us

In [21]:
model = tf.keras.models.load_model('Early_Stop_Model.h5')
mse_test = model.evaluate(X_test, y_test)

129/129 [==============================] - 0s 411us/step - loss: 0.5473 - mean_absolute_error: 0.5318
